<a href="https://colab.research.google.com/github/greyhound101/shopee/blob/main/distilbert_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
pip install sentencepiece

In [ ]:
pip install transformers

In [ ]:
import tokenization

In [ ]:
import os
import copy
import math
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import transformers
from transformers import (BertTokenizer, BertModel,
                          DistilBertTokenizer, DistilBertModel)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [ ]:
import tensorflow_hub as hub
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
# def read_and_preprocess():
#     df = pd.read_csv('/content/gdrive/My Drive/shopee/train.csv.zip')
#     tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
#     df['matches'] = df['label_group'].map(tmp)
#     df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
#     encoder = LabelEncoder()
#     df['label_group'] = encoder.fit_transform(df['label_group'])
#     N_CLASSES = df['label_group'].nunique()
#     print(f'We have {N_CLASSES} classes')
#     x_train, x_val, y_train, y_val = train_test_split(df[['title']], df['label_group'], shuffle = True, stratify = df['label_group'], random_state = SEED, test_size = 0.33)
#     return df, N_CLASSES, x_train, x_val, y_train, y_val



import numpy as np
import pandas as pd
from sklearn.preprocessing import *
def fold(idx,train):
    a=np.load( '/content/gdrive/My Drive/shopee/train_'+str(idx)+'.npy')
    df=pd.read_csv('/content/gdrive/My Drive/shopee/train.csv')
    le=LabelEncoder()
    df['label_group']=le.fit_transform(df['label_group'])
    train['label_group']=le.fit_transform(train['label_group'])
    df=df.drop_duplicates('label_group').reset_index(drop=True)
    return train.loc[train['label_group'].isin(list(df.loc[a]['label_group'].values))].reset_index(drop=True),train.loc[~train['label_group'].isin(list(df.loc[a]['label_group'].values))].reset_index(drop=True)
df=pd.read_csv('/content/gdrive/My Drive/shopee/train.csv')
train,test=fold(2,df)



In [ ]:
class CFG:
    DistilBERT = True # if set to False, BERT model will be used
    bert_hidden_size = 768
    
    batch_size = 65
    epochs = 30
    num_workers = 2
    learning_rate = 1e-5 #3e-5
    scheduler = "ReduceLROnPlateau"
    step = 'epoch'
    patience = 2
    factor = 0.8
    dropout = 0.5
    model_path = "/kaggle/working"
    max_length = 30
    model_save_name = "model.pt"
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
if CFG.DistilBERT:
    model_name='cahya/distilbert-base-indonesian'
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    bert_model = DistilBertModel.from_pretrained(model_name)
else:
    model_name='cahya/bert-base-indonesian-522M'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)

In [ ]:
text = train['title'].values[np.random.randint(0, len(train) - 1, 1)[0]]
print(f"Text of the title: {text}")
encoded_input = tokenizer(text, return_tensors='pt')
print(f"Input tokens: {encoded_input['input_ids']}")
decoded_input = tokenizer.decode(encoded_input['input_ids'][0])
print(f"Decoded tokens: {decoded_input}")
output = bert_model(**encoded_input)
print(f"last layer's output shape: {output.last_hidden_state.shape}")

Text of the title: NIVEA Hijab Fresh Whitening Deodorant Roll-On 50ml
Input tokens: tensor([[    3, 22502,  2084, 25508,  1013, 28096,  9048,  6967,  1968,  7284,
          4596,  1009, 12880,    16,  2692,  3617, 20497,     1]])
Decoded tokens: [CLS] nivea hijab fresh whitening deodorant roll - on 50ml [SEP]
last layer's output shape: torch.Size([1, 18, 768])


In [ ]:


lbl_encoder = LabelEncoder()
train['label_code'] = lbl_encoder.fit_transform(train['label_group'])
NUM_CLASSES = train['label_code'].nunique()



In [ ]:

lbl_encoder = LabelEncoder()
test['label_code'] = lbl_encoder.fit_transform(test['label_group'])



In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, mode="train", max_length=None):
        self.dataframe = dataframe
        if mode != "test":
            self.targets = dataframe['label_code'].values
        texts = list(dataframe['title'].apply(lambda o: str(o)).values)
        self.encodings = tokenizer(texts, 
                                   padding=True, 
                                   truncation=True, 
                                   max_length=max_length)
        self.mode = mode
        
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(values[idx]) for key, values in self.encodings.items()}
        if self.mode != "test":
            item['labels'] = torch.tensor(self.targets[idx]).long()
        return item
    
    def __len__(self):
        return len(self.dataframe)
dataset = TextDataset(train.sample(1000), tokenizer, max_length=CFG.max_length)
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=CFG.batch_size, 
                                         num_workers=CFG.num_workers, 
                                         shuffle=True)
batch = next(iter(dataloader))
print(batch['input_ids'].shape, batch['labels'].shape)

torch.Size([65, 30]) torch.Size([65])


In [ ]:
dataset = TextDataset(train.sample(1000), tokenizer, max_length=CFG.max_length)
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=CFG.batch_size, 
                                         num_workers=CFG.num_workers, 
                                         shuffle=True)
batch = next(iter(dataloader))
print(batch['input_ids'].shape, batch['labels'].shape)

torch.Size([65, 30]) torch.Size([65])


In [ ]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CFG.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output
class Model(nn.Module):
    def __init__(self, 
                 bert_model, 
                 num_classes=NUM_CLASSES, 
                 last_hidden_size=CFG.bert_hidden_size):
        
        super().__init__()
        self.bert_model = bert_model
        self.arc_margin = ArcMarginProduct(256, 
                                           num_classes, 
                                           s=30.0, 
                                           m=0.70, 
                                           easy_margin=False)
    
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(768,256)
        self.bn = nn.BatchNorm1d(256)
    def get_bert_features(self, batch):
        output = self.bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        last_hidden_state = output.last_hidden_state # shape: (batch_size, seq_length, bert_hidden_dim)
        CLS_token_state = last_hidden_state[:, 0, :] # obtaining CLS token state which is the first token.
        return CLS_token_state
    
    def forward(self, batch):
        CLS_hidden_state = self.get_bert_features(batch)
        x = self.dropout(CLS_hidden_state)
        x = self.fc(x)
        x = self.bn(x)
        output = self.arc_margin(x, batch['labels'])
        return output,CLS_hidden_state
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()
    
    def reset(self):
        self.avg, self.sum, self.count = [0]*3
    
    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count
    
    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def one_epoch(model, 
              criterion, 
              loader,
              optimizer=None, 
              lr_scheduler=None, 
              mode="train", 
              step="batch"):
    
    loss_meter = AvgMeter()
    acc_meter = AvgMeter()
    
    tqdm_object = tqdm(loader, total=len(loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items()}
        preds,_ = model(batch)
        loss = criterion(preds, batch['labels'])
        if mode == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if step == "batch":
                lr_scheduler.step()
                
        count = batch['input_ids'].size(0)
        loss_meter.update(loss.item(), count)
        
        accuracy = get_accuracy(preds.detach(), batch['labels'])
        acc_meter.update(accuracy.item(), count)
        if mode == "train":
            tqdm_object.set_postfix(train_loss=loss_meter.avg, accuracy=acc_meter.avg, lr=get_lr(optimizer))
        else:
            tqdm_object.set_postfix(valid_loss=loss_meter.avg, accuracy=acc_meter.avg)
    
    return loss_meter, acc_meter

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

def get_accuracy(preds, targets):
    """
    preds shape: (batch_size, num_labels)
    targets shape: (batch_size)
    """
    preds = preds.argmax(dim=1)
    acc = (preds == targets).float().mean()
    return acc
def train_eval(epochs, model, train_loader,test,
               criterion, optimizer, lr_scheduler=None):
    
    best_loss = float('inf')
    best_model_weights = copy.deepcopy(model.state_dict())
    best_val=0
    best_epoch=0
    for epoch in range(epochs):
        print("*" * 30)
        print(f"Epoch {epoch + 1}")
        current_lr = get_lr(optimizer)
        
        model.train()
        train_loss, train_acc = one_epoch(model, 
                                          criterion, 
                                          train_loader, 
                                          optimizer=optimizer,
                                          lr_scheduler=lr_scheduler,
                                          mode="train",
                                          step=CFG.step)   
#         if epoch%5==0:
#             image_embeddings = []
#             tqdm_object = tqdm(valid_loader, total=len(valid_loader))
#             for batch in tqdm_object:
#                 batch = {k: v.to(CFG.device) for k, v in batch.items()}
#                 image_embeddings.append(bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).last_hidden_state[:,0,:].detach().cpu())
        
#             image_embeddings=torch.cat(image_embeddings).numpy()
#             KNN=50
#             knn = NearestNeighbors(n_neighbors = KNN,metric='cosine')
#             knn.fit(image_embeddings)
#             distances, indices = knn.kneighbors(image_embeddings)
    
#             thresholds = list(np.arange(0,1.0, 0.01))
#             scores = []
#             best_s=0
#             best_e=0
#             best_t=0
#             for en,threshold in enumerate(thresholds):
#                 predictions = []
#                 for k in range(image_embeddings.shape[0]):
#                     idx = np.where(distances[k,] < threshold)[0]
#                     ids = indices[k,idx]
#                     posting_ids = ' '.join(test['posting_id'].iloc[ids].values)
#                     predictions.append(posting_ids)
#                 test['pred_matches'] = predictions
#                 test['f1'] = f1_score(test['matches'], test['pred_matches'])
#                 score = test['f1'].mean()
#                 scores.append(score)
#                 if best_s<score:
#                     best_s=score
#                     best_e=en
#                     best_t=threshold
#                 if en-best_e>10:
#                     break
# #             thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
# #             max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
# #             best_threshold = max_score['thresholds'].values[0]
# #             best_score = max_score['scores'].values[0]
#             print(f'Our best score is {best_s} and has a threshold {best_t}')
#             if best_val<best_s:
#                 best_epoch=epoch
#                 best_val=best_s
#             if epoch-best_epoch>10:
#                 break
#             del knn, distances, indices
#             gc.collect()
        if epoch==30:
          torch.save(model.state_dict(),'/content/gdrive/My Drive/weights_fold2_30.hdf5')
        elif epoch==50:
          torch.save(model.state_dict(),'/content/gdrive/My Drive/weights_fold2_50.hdf5')
        elif epoch==70:
          torch.save(model.state_dict(),'/content/gdrive/My Drive/weights_fold2_70.hdf5')
        torch.save(model.state_dict(),'/content/gdrive/My Drive/weights_fold2_100.hdf5')

train_dataset = TextDataset(train, tokenizer, max_length=CFG.max_length)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=CFG.batch_size, 
                                           num_workers=CFG.num_workers, 
                                           shuffle=True)

valid_dataset = TextDataset(test, tokenizer, max_length=CFG.max_length)
valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                                           batch_size=CFG.batch_size, 
                                           num_workers=CFG.num_workers, 
                                           shuffle=False)

model = Model(bert_model).to(CFG.device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG.learning_rate)
if CFG.scheduler == "ReduceLROnPlateau":
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                              mode="min", 
                                                              factor=CFG.factor, 
                                                              patience=CFG.patience)
tmp = test.groupby(['label_group'])['posting_id'].unique().to_dict()
test['matches'] = test['label_group'].map(tmp)
test['matches'] = test['matches'].apply(lambda x: ' '.join(x))
train_eval(100, model, train_loader,test,
           criterion, optimizer, lr_scheduler=lr_scheduler)



******************************
Epoch 1



******************************
Epoch 2



******************************
Epoch 3



******************************
Epoch 4



******************************
Epoch 5


...


******************************
Epoch 96



******************************
Epoch 97



******************************
Epoch 98



******************************
Epoch 99



******************************
Epoch 100
